# AEROSP 520 Compressible Flow: Homework 2
Jason Chen

## Problem 3
Consider a gas mixture composed of 1% (by weight) molecular oxygen in an argon bath at an initial equilibrium condition (1) at a pressure and temperature of $P_1$ = 1 kPa and $T_1$ = 235.6 K, respectively. The gas mixture is initially at rest in a long tube until a shock wave moving at $W$ = 2000 m/s sweeps through it.

### Part (a)
To find how long it takes for the mixture to reach a state within 1% of the final equilibrium, we note that $\tau_v$ is the amount of time it takes for the state at the beginning of a vibrational relaxation process to reach $1/e$ of the original $E_v - E^*_v$, which is a metric for the "degree of relaxation" (since $E_v - E^*_v$ approaches zero at equilibrium). From lecture:

$$
p \tau_v = Ae^{\left(\frac{B}{T}\right)^{1/3}}
$$
We know oxygen to argon collisions are going to dominate (as opposed to oxygen-oxygen) since there is physically a lot more argon, and because the oxygen to argon collisions are less efficient at transferring energy into the vibrational modes. Thus, given this thermal bath assumption, we will only evaluate the relaxation time scale for coefficients of oxygen-argon collisions (as provided in lecture). We first solve for an initial $(p,T)$ state immediately after the shock (see Part (b) below), solve for $\tau_v$ in the equation above, and then determine how many multiples of $1/e$ it takes to reach 1% of the final equilibrium.

In [25]:
A = 3.58e-4 * 1e5
B = 2.95e6
p, T = 60921.8, 3797.6
tau_v = (A * np.exp((B/T)**(1/3))) / p
# Determine how many multiples of 1/e it takes to reach 1% (note the change of base)
n = np.log(0.01) / np.log(1/np.e)
print(f"Time to reach 1% of final equilibrium: {n*tau_v:.2f} microseconds")

Time to reach 1% of final equilibrium: 26.59 microseconds


### Part (b)
What is the temperature and pressure of the mixture right after the shock?

We are given the unsteady shock wave relations, relating the speed of the wave $W$ to $\gamma$ and pressure ratio $p_1/p_2$, and the temperature and pressure rise across the shock. If we rearrange them, we can find the $p, T$ right after the shock:

$$
\frac{p_2}{p_1} = \left[ \left( \frac{W}{a_1} \right)^2 - 1 \right] \left( \frac{2\gamma}{\gamma+1} \right) + 1
$$
where $a_1 = \sqrt{\gamma R T_1}$. We can compute the gas constant of the mixture using the molecular weights of argon and oxygen and computing a weighted average by mass. The $\gamma$ can be found easily given a calorically perfect gas assumption at the initial conditions (which is reasonable given the low pressure and moderate temperature). Argon is a monoatomic gas and so $\gamma = 5/3$, while oxygen is a diatomic gas and so $\gamma = 7/5$.

$$
R_{\textrm{mix}} = R_{\textrm{Ar}}(99 \%) + R_{\textrm{O2}}(1 \%) = 208.37 \textrm{ J/g/K}
$$
$$
\gamma_{\textrm{mix}} = \gamma_{\textrm{Ar}}(99 \%) + \gamma_{\textrm{O2}}(1 \%) = 1.664
$$
Thus, $a_1 = 285.8 \textrm{ m/s}$. Thus we can get the pressure ratio from the arranged equation above, and setting the temperature ratio from the provided formula, we get:
$$
\boxed{P_2 = 60.9 \textrm{ kPa, } T_2 = 3798 \textrm{ K}}
$$

In [26]:
import numpy as np
W = 2000
p1 = 1e3
T1 = 235.6
Ru = 8.314
R_mix = (Ru/40) * 0.99 + (Ru/32) * 0.01
gamma = (5/3) * 0.99 + (7/5) * 0.01
a_1 = np.sqrt(gamma * R_mix * 1e3 * T1)
print(f"Ratio of specific heats: {gamma:.3f} | Local speed of sound = {a_1:.1f} m/s")
p2_p1 = ((W/a_1)**2 - 1) * (2*gamma/(gamma+1)) + 1
T2_T1 = p2_p1 * (((gamma+1)/(gamma-1) + p2_p1)/(1 + (gamma+1)/(gamma-1)*p2_p1))
print(f"p2: {p2_p1*p1:.1f} Pa | T2: {T2_T1*T1:.1f} K")

Ratio of specific heats: 1.664 | Local speed of sound = 285.8 m/s
p2: 60921.8 Pa | T2: 3797.6 K


### Part (c)
To find the equilibrium state, we do not necessarily need to model the relaxation process, but rather we can solve for the end state given a couple assumptions. First, we know that right after the shock, all energy is stored in the translational and rotational modes. This total energy is constant through until the equilibrium point due to conservation of energy. We are making an assumption here that the relaxation time scale for translation and rotation is much faster than the flow time scale, and that overall equilibrium can be reached. At equilibrium, we know that some of the internal energy will be stored in the vibrational mode, which is only present for oxygen. So:

$$
e_{\textrm{total}} = e_{\textrm{tr}} + e_{\textrm{rot}} + e_{\textrm{vib}} = \left(\frac{5}{2}R_{\textrm{O2}}T_2\right)(0.01) + \left(\frac{3}{2}R_{\textrm{Ar}}T_2\right)(0.99)
$$
This first equation uses the $T_2$ found in Part (b), and finds the translational + rotational modes contributed by oxygen and gives it a weight, and then sums it with the contribution from argon. Then, we can express the internal energy contribution of each species but with the vibrational mode included. Note that argon cannot have a vibrational mode because it is monoatomic.
$$
e_{\textrm{O2}}(T) = \frac{5}{2}RT + \frac{\Theta_v R}{e^{\Theta_v/T}-1}
$$
$$
e_{\textrm{Ar}}(T) = \frac{3}{2}RT
$$
Thus at equilibrium (we will denote this as state (4) to stay consistent with lecture), we can solve for temperature. We can only do this because there is only one notion of temperature (Boltzmann distribution, and at equilibrium):
$$
e_{\textrm{total}} = e_{\textrm{O2}}(0.01) + e_{\textrm{Ar}}(0.99) = \left( \frac{5}{2}R_{\textrm{O2}}T_4 + \frac{\Theta_v R_{\textrm{O2}}}{e^{\Theta_v/T_4}-1} \right)(0.01) + \frac{3}{2}R_{\textrm{Ar}}T_4 (0.99)
$$
We know the characteristic vibrational temperature $\Theta_v$ for oxygen from lecture is 2270 K. Solving for $T_4$ numerically, since an analytical solution is difficult to express:

In [27]:
from scipy.optimize import least_squares
R_o2, R_ar, theta_v = (Ru/32)/1e3, (Ru/40)/1e3, 2270
T2 = T2_T1 * T1
e_total = ((5/2) * R_o2 * T2) * 0.01 + ((3/2) * R_ar * T2) * 0.99
def function(T):
    T = T[0]
    eq1 = ((5/2)*R_o2*T + (theta_v * R_o2)/(np.exp(theta_v/T)-1))*(0.01) + ((3/2)*R_ar*T)*(0.99) - e_total
    return [eq1]
result = least_squares(function, x0=[T2])
print(f"T4: {result.x[0]:.1f} K")

T4: 3774.9 K


We can see that this relaxed temperature is within 1% of the temperature immediately downstream of the shock, and this is expected; in a thermal bath, it is by design that the relaxation will keep the temperature essentially constant since the proportion of molecules able to store energy in the vibrational modes is so small. 

From this result, it is thus also valid to say that $P_2 \approx P_4$, so we will say that $P_4 = 61 \textrm{ kPa}$. To find the speed of the wave, we can simply say that the speed of the wave must equal to the speed of the piston, which is given by the equation for $u_p$ given in the problem statement, due to conservation of momentum.

In [28]:
u_p = (a_1/gamma) * (p2_p1-1) * np.sqrt(((2*gamma)/(gamma+1)) / (p2_p1 + (gamma-1)/(gamma+1)))
print(f"Speed of the wave: {u_p:.1f} m/s")

Speed of the wave: 1470.8 m/s


So the final answer (because of the thermal bath assumption):
$$
\boxed{T_4 = 3798 \textrm{ K, } P_4 = 61 \textrm{ kPa, } u_{\textrm{wave}} = 1471 \textrm{ m/s}}
$$

### Part (d)
The answer changes significantly for a 50/50 mixture of oxygen and argon because the thermal bath assumption no longer holds.